In [1]:
import definitions
import pandas_helper
import scikit_helper
import lendingclub_helper


In [2]:
definitions.dataFiles

{'2007-2011': '../data/LoanStats3a_securev1.csv',
 '2012-2013': '../data/LoanStats3b_securev1.csv',
 '2014': '../data/LoanStats3c_securev1.csv',
 '2015': '../data/LoanStats3d_securev1.csv',
 '2016Q1': '../data/LoanStats_securev1_2016Q1.csv',
 '2016Q2': '../data/LoanStats_securev1_2016Q2.csv',
 '2016Q3': '../data/LoanStats_securev1_2016Q3.csv',
 '2016Q4': '../data/LoanStats_securev1_2016Q4.csv',
 '2017Q1': '../data/LoanStats_securev1_2017Q1.csv',
 '2017Q2': '../data/LoanStats_securev1_2017Q2.csv',
 '2017Q3': '../data/LoanStats_securev1_2017Q3.csv',
 '2017Q4': '../data/LoanStats_securev1_2017Q4.csv'}

In [3]:
key = '2007-2011'
datafile = definitions.dataFiles[key]
df = pandas_helper.readData(datafile)

In [4]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,1077501,NaN,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,NaN,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,NaN,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,NaN,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,NaN,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
lendingclub_helper.buildFeatures(df)

In [8]:
lendingclub_helper.getFeatureColumns(df)

Index(['f_acc_now_delinq', 'f_acc_open_past_24mths', 'f_addr_state_AK',
       'f_addr_state_AL', 'f_addr_state_AR', 'f_addr_state_AZ',
       'f_addr_state_CA', 'f_addr_state_CO', 'f_addr_state_CT',
       'f_addr_state_DC',
       ...
       'f_total_rec_prncp', 'f_total_rev_hi_lim', 'f_revol_bal_joint',
       'f_hardship_amount', 'f_orig_projected_additional_accrued_interest',
       'f_hardship_payoff_balance_amount', 'f_hardship_last_payment_amount',
       'f_settlement_amount', 'f_settlement_percentage', 'f_settlement_term'],
      dtype='object', length=134)